In [1]:
import pandas as pd

In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome(options=options)

In [5]:
import selenium 

In [7]:
from selenium.webdriver.support.ui import Select

In [9]:
from selenium.webdriver.support.ui import Select, WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [11]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [13]:
browser = webdriver.Chrome(options=options)
browser.get("https://www.mse.mk/mk/stats/symbolhistory/ADIN")

In [15]:
options_tmp = browser.find_elements(By.CSS_SELECTOR,"#Code > option")

In [17]:
print(len(options_tmp))

377


In [19]:
import time

# BS4

In [22]:
from bs4 import BeautifulSoup as bs
import requests

In [48]:
import numpy as np

In [275]:
import datetime

In [319]:
import warnings

In [24]:
url = "https://www.mse.mk/mk/stats/symbolhistory/ADIN"
response = requests.get(url)
html_content = response.content 
soup = bs(html_content,'html.parser')

In [26]:
options_tmp = soup.select("#Code > option")
options_list = []
for option in options_tmp:
    if any(char.isdigit() for char in option.text):
        continue
    else:
        options_list.append(option.text)

In [249]:
timestamp_csv = pd.DataFrame(columns=['Code', 'Date'])
timestamp_csv.to_csv("timestamps/timestamps.csv", index=False)

In [372]:
if(len(timestamp_csv)!=options_list):
    first_search()
else:
    update_database()

KeyboardInterrupt: 

In [370]:
def first_search():
    timestamps = pd.DataFrame(columns=['Code', 'Date'])
    base_url ="https://www.mse.mk/mk/stats/symbolhistory/{}"
    opts_test = options_list[:2]
    for opt in options_list:
        url_opt = base_url.format(opt)
        browser.get(url_opt)
        year = 2014
        now = pd.Timestamp.now()
        mntDayFrom=f"{now.day}.{now.month}."
        mntDayTo = mntDayTo = f"{now.day - 1}.{now.month}." if now.day > 1 else f"{(now.day - 1 + 30) % 30}.{(now.month - 1) % 12}."
        final_table_for_option = []
        for i in range(0,10):
            browser.get(url_opt)
            code = Select(browser.find_element(By.CSS_SELECTOR,"#Code"))
            btn =browser.find_element(By.CSS_SELECTOR,"#report-filter-container > ul > li.container-end > input")
            from_date = browser.find_element(By.CSS_SELECTOR,"#FromDate")
            to_date = browser.find_element(By.CSS_SELECTOR,"#ToDate")
            from_date.clear()
            from_date.send_keys(mntDayFrom + str(year))
            to_date.clear()
            to_date.send_keys(mntDayTo + str(year + 1))
            code.select_by_value(opt)
            btn.click()
            
            resp = requests.get(url_opt)
            soup_i = bs(browser.page_source,"html.parser")      
            opt_table_rows = soup_i.select("#resultsTable > tbody > tr")
            opt_table_rows = opt_table_rows[::-1]
            for row in opt_table_rows:
                row_cells = row.select("td")
                row_cells_text = []
                for cell in row_cells:
                    row_cells_text.append(cell.text)
                date= row_cells[0].text
                month= date[3:5]
                year_str = date[6:]
                year_str="20"+year_str[-2:]
                # print(row.find_element(By.CSS_SELECTOR, "td:nth-child(2)").text)
                dic ={
                    "Date": date,
                    "Year": year_str,
                    "Month": month,
                    "Price for last transaction": row_cells[1].text,
                    "Maximum": row_cells[2].text,
                    "Minimum": row_cells[3].text,
                    "Average price": row_cells[4].text,
                    "%prom": row_cells[5].text,
                    "Quantity": row_cells[6].text,
                    "Traffic BEST mkd": row_cells[7].text,
                    "Total traffic": row_cells[8].text
                }
                final_table_for_option.append(dic)
            year= year+1
        df_i = pd.DataFrame(final_table_for_option)
        df_i.to_csv(opt+".csv", encoding="utf-8-sig", index=False)
        tmst_dict = {
            "Code" : opt,
            "Date": pd.Timestamp.now().date()
        }
        print(tmst_dict)
        timestamps = timestamps._append(tmst_dict, ignore_index=True)
        print(opt)
    update_timestamps(timestamps)

In [360]:
def update_timestamps(timest):
    try:
        timestamp_csv = pd.read_csv("timestamps/timestamps.csv")
        print("Successfully loaded existing CSV.")
    except FileNotFoundError:
        print("CSV file not found, creating new DataFrame.")
        timestamp_csv = pd.DataFrame(columns=['Code', 'Date'])
        
    timestamp_csv['Date'] = pd.to_datetime(timestamp_csv['Date'], errors='coerce')
    timest['Date'] = pd.to_datetime(timest['Date'], errors='coerce')
    
    timest = timest.set_index(['Code', 'Date'])
    timestamp_csv = timestamp_csv.set_index(['Code', 'Date'])
    
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=UserWarning)
        m = timestamp_csv.combine_first(timest)
    
    m.reset_index(inplace=True)
    m = m.drop_duplicates(subset=['Code'], keep='first')
    
    m.to_csv("timestamps/timestamps.csv", index=False)
    print("Updated CSV saved successfully.")


In [366]:
def update_database():
    try:
        timestamp_csv = pd.read_csv("timestamps/timestamps.csv")
        timestamp_csv['Date'] = pd.to_datetime(timestamp_csv['Date'])
    except FileNotFoundError:
        print("CSV file not found. Make sure the file exists and try again.")
        return
    
    base_url = "https://www.mse.mk/mk/stats/symbolhistory/{}"
    now = pd.Timestamp.now()
    new_data = []

    for _, row in timestamp_csv.iterrows():
        opt = row['Code']
        last_date = row['Date']
        url_opt = base_url.format(opt)
        from_date_str = last_date.strftime("%d.%m.%Y")
        to_date_str = now.strftime("%d.%m.%Y")

        browser.get(url_opt)
        code = Select(browser.find_element(By.CSS_SELECTOR, "#Code"))
        btn = browser.find_element(By.CSS_SELECTOR, "#report-filter-container > ul > li.container-end > input")
        from_date = browser.find_element(By.CSS_SELECTOR, "#FromDate")
        to_date = browser.find_element(By.CSS_SELECTOR, "#ToDate")
        from_date.clear()
        from_date.send_keys(from_date_str)
        to_date.clear()
        to_date.send_keys(to_date_str)
        code.select_by_value(opt)
        btn.click()

        resp = requests.get(url_opt)
        soup_i = bs(browser.page_source, "html.parser")
        opt_table_rows = soup_i.select("#resultsTable > tbody > tr")
        opt_table_rows = opt_table_rows[::-1]
        for row in opt_table_rows:
            row_cells = row.select("td")
            row_cells_text = [cell.text for cell in row_cells]
            dic = {
                "Code": opt,
                "Date": row_cells[0].text,
                "Year": "20" + row_cells[0].text[-2:],
                "Month": row_cells[0].text[3:5],
                "Price for last transaction": row_cells[1].text,
                "Maximum": row_cells[2].text,
                "Minimum": row_cells[3].text,
                "Average price": row_cells[4].text,
                "%prom": row_cells[5].text,
                "Quantity": row_cells[6].text,
                "Traffic BEST mkd": row_cells[7].text,
                "Total traffic": row_cells[8].text
            }
            new_data.append(dic)

    if not new_data:
        print("No new data found.")
        return

    new_df = pd.DataFrame(new_data)
    
    for opt in options_list:
        try:
            existing_df = pd.read_csv(opt + ".csv")
            updated_df = pd.concat([existing_df, new_df[new_df['Code'] == opt]])
        except FileNotFoundError:
            updated_df = new_df[new_df['Code'] == opt]

        updated_df.to_csv(opt + ".csv", encoding="utf-8-sig", index=False)

    timestamps = pd.DataFrame(columns=['Code', 'Date'])
    for opt in options_list:
        tmst_dict = {
            "Code": opt,
            "Date": pd.Timestamp.now().date()
        }
        timestamps = timestamps._append(tmst_dict, ignore_index=True)

    timestamps.to_csv("timestamps/timestamps.csv", index=False)
    print("Database updated successfully.")
